In [15]:
import pandas as pd
import numpy as np
from scipy.optimize import bisect
from scipy.stats import norm

file_path = './SPX_options.csv'
data = pd.read_csv(file_path)

In [16]:
data.describe()

,S,Dividend,C_BS,D_BS,C_mkt,D_Blm,R,TTM,Moneyness,D_Optimal,K
count,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.00000,85.000000,85.000000,85.000000,85.0
mean,5718.486941,1.345927,300.508027,0.591168,315.221176,0.609341,4.80564,168.870588,18.458824,0.060418,5700.0
std,201.911555,0.047218,93.940648,0.120690,95.263399,0.120187,0.10632,35.091305,201.966260,5.258367,0.0
min,5186.330000,1.276500,106.223538,0.291429,111.850000,0.312000,4.59072,108.000000,-514.000000,-39.819644,5700.0
25%,5597.120000,1.311000,237.004250,0.526772,257.450000,0.549000,4.76095,140.000000,-103.000000,0.424070,5700.0
50%,5728.800000,1.336400,295.415563,0.591379,311.800000,0.610000,4.77997,169.000000,29.000000,0.580349,5700.0
75%,5853.980000,1.372100,368.601090,0.669328,383.750000,0.686000,4.88116,198.000000,154.000000,0.765608,5700.0
max,6049.880000,1.481600,466.011822,0.800804,487.300000,0.818000,5.01499,228.000000,350.000000,8.794966,5700.0


In [17]:
# Black-Scholes call option pricing formula
def black_scholes_call_price(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

# Function to calculate implied volatility
def implied_volatility(C_mkt, S, K, T, r):
    def objective(sigma):
        return black_scholes_call_price(S, K, T, r, sigma) - C_mkt
    try:
        return bisect(objective, 1e-6, 5)  # Searching for sigma in a reasonable range
    except ValueError:
        return np.nan

# Function to calculate Delta using the Black-Scholes model
def black_scholes_delta(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return norm.cdf(d1)

# Crank-Nicholson method for solving the Black-Scholes PDE
def crank_nicholson(S, K, r, T, sigma, N=100, M=100):
    dt = T / N  # Time step size
    dx = sigma * np.sqrt(3 * dt)  # Price step size
    pu = 0.5 * dt * ((sigma / dx)**2 + (r - 0.5 * sigma**2) / dx)
    pm = 1 - dt * (sigma / dx)**2 - r * dt
    pd = 0.5 * dt * ((sigma / dx)**2 - (r - 0.5 * sigma**2) / dx)

    # Initialize the price grid
    S_grid = S * np.exp(dx * (np.arange(-M, M+1)))
    # print(S_grid)
    V = np.maximum(S_grid - K, 0)  # Option value at maturity

    # Backward iteration to solve the PDE
    for _ in range(N):
        V[1:-1] = pu * V[:-2] + pm * V[1:-1] + pd * V[2:]
        V[0] = 0  # Boundary condition at S = 0
        V[-1] = S_grid[-1] - K  # Boundary condition at S -> infinity
    return V[M]



# In[ ]:






In [18]:
# Initialize strategy variables
position = 0  # Current asset position
cash = 0      # Cash balance
portfolio_values = []  # Store portfolio values over time
predicted_prices = []  # Store predicted option prices

# Iterate over each row of the data to implement the hedging strategy
for i in range(len(data)):
    S = data['S'][i]  # Current stock pricex
    K = data['K'][i]  # Calculate strike price based on Moneyness
    T = data['TTM'][i] / 252  # Convert time to expiration to years
    r = data['R'][i] / 100  # Convert interest rate to decimal
    C_mkt = data['C_mkt'][i]  # Market option price

    # Calculate implied volatility
    sigma = implied_volatility(C_mkt, S, K, T, r)

    # Skip this row if implied volatility could not be calculated
    if np.isnan(sigma):
        continue

    # Calculate Delta using Crank-Nicholson if needed
    V_up = crank_nicholson(S * 1.001, K, r, T, sigma, 1000)
    V_down = crank_nicholson(S * 0.999, K, r, T, sigma, 1000)
    delta = (V_up - V_down) / (S * 0.02)

    # Calculate predicted option price using Crank-Nicholson
    predicted_price = crank_nicholson(S, K, r, T, sigma)
    predicted_prices.append(predicted_price)

    # Determine the target position based on Delta
    target_position = -delta
    position_change = target_position - position

    # Update cash and asset position
    cash -= position_change * S
    position = target_position

    # Record portfolio value
    portfolio_values.append(position * S + cash)

In [23]:
# Convert portfolio values to a DataFrame for further analysis
portfolio_values = pd.DataFrame(portfolio_values, columns=['Portfolio Value'])

# Print predicted prices
predicted_prices_df = pd.DataFrame(predicted_prices, columns=['Predicted Price'])
# print(predicted_prices_df.head())
# print("---------------")
# print(data.head()['C_mkt'])
print(((data['C_mkt'] - predicted_prices)**2).mean())

# Calculate risk metrics
portfolio_values['Returns'] = portfolio_values['Portfolio Value'].pct_change().dropna()
volatility = portfolio_values['Returns'].std()  # Calculate return volatility
cumulative_returns = (1 + portfolio_values['Returns']).cumprod()
drawdown = cumulative_returns.cummax() - cumulative_returns  # Calculate drawdown
max_drawdown = drawdown.max()
# Output risk metrics
volatility, max_drawdown
# print("_________")
# print(volatility, max_drawdown)

28926.356711593566


/Users/jacksparrow/NYU_schoolwork/2024/quant_methods/Crank-Nicholson_delta_hedging/venv/lib/python3.13/site-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


(np.float64(nan), np.float64(nan))